# Crime Reports Database Project

The goal of this project is to create a postgre database and set up permissions for two groups one with read only permissions for the data analysts and one with read write permissions for the data scientists

In [1]:
# importing libraries and creating connection and cursor python objects
import psycopg2
conn = psycopg2.connect(dbname='dq', user='dq')
cur = conn.cursor()

In [2]:
# creating crimes database
conn.autocommit = True
cur.execute("CREATE DATABASE crimes_db OWNER dq;")
conn.close()

In [3]:
# connecting to new crimes_db and creating a crimes schema
conn = psycopg2.connect(dbname='crimes_db', user='dq')
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")
conn.commit()

In [4]:
# opening bost.csv and viewing column headers and first row
import csv
with open("boston.csv") as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)
    
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [5]:
# function to check the total number of unique values in a column
def get_col_set(csv_filename, col_index):
    with open(csv_filename, 'r') as f:
        next(f)
        reader = csv.reader(f)
        values = set()
        for row in reader:
            values.add(row[col_index])
        return values            

In [6]:
# loop to run through each column and run the unique values function
for i in range(len(col_headers)):
    col_name = col_headers[i]
    total_values = len(get_col_set("boston.csv", i))
    print("{}: {}".format(col_name, total_values))

incident_number: 298329
offense_code: 219
description: 239
date: 1177
day_of_the_week: 7
lat: 18177
long: 18177


In [7]:
# creating set for the description column
description_set = get_col_set("boston.csv", 2)

In [8]:
# finding max length value in description column set
len_max = 0
max_index = 0
count =-1
for i in description_set:
    length = len(i)
    count+=1
    len_max = max(length,len_max)
    if len_max == length:
        max_index=count  

In [9]:
dscrp_list = list(description_set)
print(len_max)
print(dscrp_list[max_index])

58
RECOVERED - MV RECOVERED IN BOSTON (STOLEN OUTSIDE BOSTON)


In [10]:
# reviewing data to create table and formats
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">incident_numer</span> - This will be the primary key as it is a unique identifier. We can use an INTEGER datatype as it goes up to 300K

<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">offense_code</span> - This will be an INTEGER as well 

<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">description</span> - Our max description length was 58 chars so we could use a VARCHAR of 75 for some wiggle room if more data is added while saving space vs using TEXT

<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">date</span> - We will use DATE type

<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">day_of_the_week</span> - We will use a enumerated data type since the values should be between 7 types (we don't want othe ways to spell a day of the week)

<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">lat</span> - We will use decimal for lat as it is a decimal number

<span style="background-color: #f0f0f0; padding: 2px 5px; border-radius: 3px;">long</span> - We will use decimal for long as it is a decimal number

In [34]:
# creating enumerated day of week column
cur.execute("CREATE TYPE day_of_week AS ENUM ('Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday')")

In [35]:
# creating column names and data types
cur.execute("""
            CREATE TABLE crimes.boston_crimes(
                incident_number INTEGER PRIMARY KEY,
                offense_code INTEGER,
                description VARCHAR(75),
                date DATE,
                day day_of_week,
                lat decimal,
                long decimal                
                );
                """
            )

In [36]:
# copying CSV into the created table
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;",f)

In [49]:
# selecting all data and confirming it loaded correctly
cur.execute("SELECT * FROM crimes.boston_crimes LIMIT 5")
cur.fetchall()

[(1,
  619,
  'LARCENY ALL OTHERS',
  datetime.date(2018, 9, 2),
  'Sunday',
  Decimal('42.35779134'),
  Decimal('-71.13937053')),
 (2,
  1402,
  'VANDALISM',
  datetime.date(2018, 8, 21),
  'Tuesday',
  Decimal('42.30682138'),
  Decimal('-71.06030035')),
 (3,
  3410,
  'TOWED MOTOR VEHICLE',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.34658879'),
  Decimal('-71.07242943')),
 (4,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.33418175'),
  Decimal('-71.07866441')),
 (5,
  3114,
  'INVESTIGATE PROPERTY',
  datetime.date(2018, 9, 3),
  'Monday',
  Decimal('42.27536542'),
  Decimal('-71.09036101'))]

In [38]:
# this revokes all priviledges for any groups or users as the public group will give them by default
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crimes_db FROM public;")

In [39]:
# this creates a readonly and readwrite group
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("CREATE GROUP readwrite NOLOGIN;")

In [40]:
# granting connection and usage access to the crimes_db for both groups
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readonly")
cur.execute("GRANT CONNECT ON DATABASE crimes_db TO readwrite")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")

In [41]:
# grants permission to each group for all tables in the crimes schema
cur.execute("GRANT INSERT, SELECT ON ALL TABLES IN SCHEMA crimes TO readonly")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite")

In [42]:
# creating users for the created db and groups
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst")

In [43]:
# creating users for the created db and groups
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist")

In [47]:
# checkiong the priviledges on readwrite account
cur.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee = 'readwrite';")
cur.fetchall()

[('readwrite', 'INSERT'),
 ('readwrite', 'SELECT'),
 ('readwrite', 'UPDATE'),
 ('readwrite', 'DELETE')]

In [48]:
# checkiong the priviledges on readonly account
cur.execute("SELECT grantee, privilege_type FROM information_schema.table_privileges WHERE grantee = 'readonly';")
cur.fetchall()

[('readonly', 'INSERT'), ('readonly', 'SELECT')]

In [33]:
# use conn.rollback() if errors happen with the kernal